Trying out scikit-learn locally on my computer with a 0.4 sample  of my original data (which was as large as I could go before running into memory issues when trying to save out a csv file with Spark / on EMR) — still a work in progress...

In [42]:
import numpy as np
import pandas as pd

import re
from bs4 import BeautifulSoup 

import nltk
from nltk.corpus import stopwords 

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
%matplotlib inline

In [43]:
meetup_raw = pd.read_csv('meetup.csv')
meetup_raw.head(2)

,description,status,event_id,event_name,category
0,<p>FOOD AND FELLOWSHIP | 5 PM</p> \n<p>CONCERT...,upcoming,239660020,We Are Messengers Concert,singles
1,<p><b>【WhyNot!?JAPAN + MeetUp Collaboration Ev...,upcoming,239719229,Friendly Friday!!,socializing


In [45]:
meetup_raw.category.value_counts()

socializing                 26063
outdoors/adventure          19167
tech                        19161
career/business             18029
health/wellbeing            17534
new age/spirituality        11082
sports/recreation            9217
fitness                      8412
language/ethnic identity     7928
games                        5370
food/drink                   5310
fine arts/culture            4235
dancing                      4224
education/learning           3783
music                        3611
movements/politics           3520
community/environment        3281
singles                      2899
religion/beliefs             2892
parents/family               2537
photography                  2153
hobbies/crafts               1981
movies/film                  1888
support                      1653
LGBT                         1628
book clubs                   1392
writing                      1387
cars/motorcycles             1338
pets/animals                 1123
sci-fi/fantasy

In [46]:
out_adv = meetup_raw.category == 'outdoors/adventure'; tech = meetup_raw.category == 'tech'
meetup_2cls = meetup_raw[out_adv | tech]
meetup_2cls.category.value_counts()

outdoors/adventure    19167
tech                  19161
Name: category, dtype: int64

In [47]:
meetup_2cls.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38328 entries, 2 to 195318
Data columns (total 5 columns):
description    36401 non-null object
status         38328 non-null object
event_id       38328 non-null object
event_name     38328 non-null object
category       38328 non-null object
dtypes: object(5)
memory usage: 1.8+ MB


In [48]:
meetup_2cls = meetup_2cls.dropna()
meetup_2cls.shape

(36401, 5)

In [49]:
meetup_2cls['category_bin'] = meetup_2cls.category.apply(lambda x: 1 if x == "tech" else 0)
meetup_2cls.head(2)

,description,status,event_id,event_name,category,category_bin
2,<p>TENTATIVE<br></p>\n<p><br></p>\n<p>10.30 - ...,upcoming,239691981,Ansible meetup,tech,1
8,<p>This is an introductory meeting to get to k...,upcoming,mtzxwmywjbjb,Authentication options,tech,1


In [50]:
meetup_2cls.reset_index(drop=True, inplace=True)
meetup_2cls = meetup_2cls[['description', 'category_bin']]
meetup_2cls.head(2)

,description,category_bin
0,<p>TENTATIVE<br></p>\n<p><br></p>\n<p>10.30 - ...,1
1,<p>This is an introductory meeting to get to k...,1


In [51]:
def text_cleaner(text):
    
    '''Cleans out html tags and other non letter characters with a regular expression.
    Removes stop words and outputs a single string.'''
    
    # strip html 
    no_html = BeautifulSoup(text).get_text() 
 
    # retain only letters        
    letters = re.sub("[^a-zA-Z]", " ", no_html) 

    # lower case & split into words
    raw_words = letters.lower().split()                             

    # convert stop words set because faster to search
    stop_words = set(stopwords.words("english"))                  

    # remove stop words
    words = [word for word in raw_words if not word in stop_words]   
    
    # join words back together
    return( " ".join(words))   

In [52]:
meetup_2cls['description'] = meetup_2cls.description.map(text_cleaner)

/Users/VanessaG/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/VanessaG/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/Users/VanessaG/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://cst.dk/anders/main.html" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_

In [53]:
meetup_2cls.head(2)

,description,category_bin
0,tentative introduction orientation session bre...,1
1,introductory meeting get know fellow coders sh...,1


In [54]:
# train test split before creating bag of words
X = meetup_2cls['description']
y = meetup_2cls['category_bin']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=.25)

In [56]:
Xtrain = Xtrain.values

In [64]:
Xtrain.shape

(27300,)

In [58]:
# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None,  
                             preprocessor = None, stop_words = None, 
                             max_features = 5000) 

In [59]:
train_features = vectorizer.fit_transform(Xtrain)

In [61]:
train_features = train_features.toarray()

In [65]:
#It has 16658 rows and 5,000 features (one for each vocabulary word).
train_features.shape

(27300, 5000)

In [67]:
vocab = vectorizer.get_feature_names()
len(vocab)

5000

In [68]:
forest = RandomForestClassifier(n_estimators = 100) 


In [69]:
forest = forest.fit(train_features, ytrain)

In [70]:
test_features = vectorizer.transform(Xtest)

In [71]:
test_features = test_features.toarray()

In [72]:
predictions = forest.predict(test_features)

In [ ]:
forest.

In [77]:
ytest.shape, ytrain.shape

((9101,), (27300,))

In [82]:
y[ytest].shape

(9101,)

In [83]:
forest.score(test_features, y[ytest])

0.47731018569387978

In [84]:
df = pd.DataFrame(data={'actual': ytest, 'predicted': predictions})

In [85]:
df

,actual,predicted
26893,1,1
15907,0,0
31285,0,0
25616,1,1
18239,0,0
34756,1,1
20385,0,1
31476,0,0
33326,1,1
34956,0,0
